# Term Project_전기차 충전소가 정말 부족할까?

(1) 주제 선정 이유: <br>
    전기차가 상용화된지 많은 시간이 지났지만 아직까지 전기차 수에 비해 전기차 충전소가 부족하다고 한다. 이것이 정말인지 일반 자동차와 비교하여 확인하고자 선정했다.

(2) 가설 정의: <br>
    일반 자동차와 주유소의 비율보다 전기차와 충전소의 비율이 더 작을 것이다.

(3) 인터넷을 통한 데이터 획득: <br>
    정부3.0에서 전국 전기차 충전소 데이터(API)를 얻는다.<br>
    정부3.0에서 한국도로공사_주유소별 가격,업체 현황 데이터(csv)를 얻는다.<br>
    정부3.0에서 국토교통부 자동차 등록 현황 데이터(excel)를 얻는다.<br>
    API사용을 위해 행정표준코드관리시스템에서 법정동코드 데이터(csv)를 얻는다.

(4) 분석을 위한 데이터의 가공: <br>
    법정동코드 데이터를 통해 폐지되지않은 법정동코드를 구한뒤 7자리의 코드중 필요한 앞부분 2개만 추출한다.<br>
    전국 전기차 충전소 데이터와 추출한 법정동코드를 통해 전국의 충전소의 개수를 파악한다.<br>
    한국도로공사_주유소별 가격,업체 현황 데이터 통해 전국의 주유소의 수를 파악한다.<br>
    국토교통부 자동차 등록 현황 데이터 통해 전국의 일반 자동차와 전기차의 차량수를 파악한다.<br>
    일반자동차의 수를 주유소의 수로 나눈 값과 전기차의 수를 충전소의 수로 나눈값과 비교한다.<br>

(5) 분석 결과 도출:<br>
전국의 일반 자동차의 수는 22702848대, 전기 자동차의 수는 272656대이고 전국의 주유소는 11402개, 전기차 충전소는 119035대가 있다는 결과가 나왔다.<br>
    이때 (주유소 수)/(자동차 수)의 식을 통해 일반 자동차 한대당 주유소는 0.003013개가 있고 전기 자동차 한대당 충전소는 0.436576개가 있다는것을 파악했다.<br>
    이렇게 주유소보다 전기차 충전소가 훨씬 작다는 결과가 나왔다.<br><br>
    하지만 이 결과는 주유소의 평균 주유 기계의 대수, 전기 자동차와 일반 자동차의 연비차이, 전기 자동차와 일반 자동차의 평균 연료 보관량의 차이를 전혀 고려하지않은 결과이다.<br>
    정확한 결과를 도출하기 위해서는 상기 언급한 자료들을 획득 및 분석을 해야하는데, 이 프로젝트에서는 상기 언급한 자료를 찾지 못하여 주유소보다 전기차 충전소가 훨씬 적다는 결과가 도출되었다.

(6) 결론:<br>
    이 프로젝트에서 자료를 획득 및 분석한 결과로는 전기차 충전소가 주유소보다 많다는 결과가 나왔다.

(7) 참고문헌:<br>
    정부3.0 API사용법 및 xml가공법: https://wonhwa.tistory.com/16<br>
    pandas 인코딩 오류 해결법: https://zephyrus1111.tistory.com/39<br>
    파이썬으로 데이터 주무르기(1판), 민형기 지음, 비제이퍼블릭 출판

(8) 별첨: (3)의 획득한 데이터 원본<br>
    'work'폴더 확인

(9) 별첨: (4)의 가공된 데이터 원본<br>
    'result'폴더 확인

(10) 별첨: (3)을 (4)로 변환하는 등의 작업을 위하여 본인이 직접 개발한 Python 소스코드 원본<br>
    

In [ ]:
import pandas as pd
import requests
import bs4
import os
from dotenv import load_dotenv

load_dotenv()

#전국 및 지역 주유소 데이터
GasStations=pd.read_csv('./work/Gas_Station.csv', encoding='cp949') #csv 읽기
GasStations.set_index('시도', inplace=True) #인덱스를 시도로 변경
Nation_GasStations=GasStations.at['전국','주유소 수']



#내연 및 전기 자동차 데이터
Cars=pd.read_excel('./work/Cars.xlsx', sheet_name='10.연료별_등록현황', usecols='A,U', header=3) #excel 읽기
Cars.rename(columns={Cars.columns[0]:'연료',Cars.columns[1]:'합계'},inplace=True) #열 이름 변경
Cars['연료'].fillna(method='ffill', inplace=True) #NAN 값을 연료 이름으로 채움
Cars.drop_duplicates(['연료'], keep='last', inplace=True) #각 연료의 마지막 부분인 총계만 남기고 제거
Cars.set_index('연료', inplace=True) #인덱스를 연료로 변경
Electric_Cars=Cars.at['전기','합계']
Internal_Combustion_Cars=Cars.at['휘발유','합계']+Cars.at['경유','합계']+Cars.at['하이브리드(휘발유+전기)','합계']+Cars.at['하이브리드(경유+전기)','합계']



#전기차 충전소 API
def getElectricStations(zcode):
    url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerInfo'
    pageNo='1' #페이지번호
    numOfRows='10' #한번에 받을 정보 개수
    serviceKey = os.environ.get("ELECTRIC_STATION_API_KEY")
    params ={'serviceKey' : serviceKey, 'pageNo' : pageNo, 'numOfRows' : numOfRows, 'zcode' : zcode }
    response = requests.get(url, params=params) #api response
    content = response.text
    
    xml_obj = bs4.BeautifulSoup(content,'lxml-xml') #totalCount 추출
    rows = xml_obj.find('totalCount')
    return int(rows.get_text())

#폐지되지 않은 법정동코드 데이터
ZcodeData=pd.read_csv('./work/법정동코드 전체자료.csv', encoding='cp949', delimiter='\t')
ZcodeData.set_index('폐지여부', inplace=True) #폐지여부를 인덱스로 설정
ZcodeData.drop(['폐지'], inplace=True) #인덱스를 사용하여 폐지된 법정동코드 삭제
L_zcode=ZcodeData['법정동코드']
R_zcode=[]
for i in L_zcode: #7자리의 법정동코드중 API에 필요한 앞의 두숫자만 추출
    R_zcode.append(str(i)[0:2])    
ZcodeData['법정동코드']=R_zcode

ZcodeData.drop_duplicates(['법정동코드'], inplace=True) #중복되는 법정동코드 삭제
zcode=ZcodeData['법정동코드']

#전기차 충전소 데이터
ElectricStations=0
for i in zcode:
    ElectricStations+=getElectricStations(i) #추출한 법정동코드를 API에 입력 및 결과값 누적
        
        
        
        
        
#결과 처리
resultOf_Electric_Cars = ElectricStations/Electric_Cars
resultOf_Internal_Combustion_Cars = Nation_GasStations*6/Internal_Combustion_Cars

#결과 저장
result_data = pd.DataFrame(data=[[Nation_GasStations,Internal_Combustion_Cars,resultOf_Internal_Combustion_Cars],[ElectricStations,Electric_Cars,resultOf_Electric_Cars]],index=['일반 자동차','전기 자동차'],columns=['주유소 수','자동차 수','주유소 수 / 자동차 수'])
print(result_data)
result_data.to_excel('./result/result.xlsx', sheet_name='result')
        